In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU
from keras.optimizers import RMSprop

# Define the generator model
def build_generator():
    model = Sequential()
    model.add(Dense(7 * 7 * 256, input_dim=100))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((7, 7, 256)))
    model.add(Conv2DTranspose(128, kernel_size=5, strides=1, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(64, kernel_size=5, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(1, kernel_size=5, strides=2, padding='same', activation='tanh'))
    return model

# Define the discriminator model
def build_discriminator():
    model = Sequential()
    model.add(Conv2D(64, kernel_size=5, strides=2, padding='same', input_shape=(28, 28, 1)))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, kernel_size=5, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Flatten())
    model.add(Dense(1))
    return model

# Define the WGAN model
def build_wgan(generator, discriminator):
    discriminator.trainable = False
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# Define the Wasserstein loss function
def wasserstein_loss(y_true, y_pred):
    return -K.mean(y_true * y_pred)

# Build the generator and discriminator models
generator = build_generator()
discriminator = build_discriminator()

# Build the WGAN model
wgan = build_wgan(generator, discriminator)

# Compile the WGAN model
wgan.compile(optimizer=RMSprop(lr=0.00005), loss=wasserstein_loss)

# Train the WGAN model
def train_wgan(x_train, epochs, batch_size):
    for epoch in range(epochs):
        for batch in range(x_train.shape[0] // batch_size):
            # Train the discriminator
            for _ in range(5):
                noise = np.random.normal(0, 1, (batch_size, 100))
                fake_images = generator.predict(noise)
                real_images = x_train[np.random.randint(0, x_train.shape[0], batch_size)]
                discriminator_loss_real = discriminator.train_on_batch(real_images, -np.ones(batch_size))
                discriminator_loss_fake = discriminator.train_on_batch(fake_images, np.ones(batch_size))
                discriminator_loss = 0.5 * np.add(discriminator_loss_real, discriminator_loss_fake)
                for layer in discriminator.layers:
                    weights = layer.get_weights()
                    weights = [np.clip(w, -0.01, 0.01) for w in weights]
                    layer.set_weights(weights)
            
            # Train the generator
            noise = np.random.normal(0, 1, (batch_size, 100))
            generator_loss = wgan.train_on_batch(noise, -np.ones(batch_size))
        
        # Print the losses
        print(f"Epoch {epoch+1}/{epochs} - Discriminator Loss: {discriminator_loss} - Generator Loss: {generator_loss}")

# Train the WGAN model
train_wgan(x_train, epochs=100, batch_size=64)
